In [1]:
import pandas as pd
import openpyxl
import re
import locale
from datetime import datetime

from selenium import webdriver
from bs4 import BeautifulSoup
import time
import os

In [2]:
# Извлечение числа просмотров и уникальных просмотров объявления
def extract_numbers(text):
    # 1. Извлекаем число просмотров
    views_pattern = re.compile(r'\d{2}:\d{2}\s*([\d\s]+)\s+просмотр(?:а|ов)?')
    views_matches = views_pattern.findall(text)
    views = int(views_matches[0].replace(' ', '')) if views_matches else None

    # 2. Извлекаем число уникальных просмотров
    unique_pattern = re.compile(r'(\d+)\s+уникальн(?:ых|ых|ые|ий)?')
    unique_matches = unique_pattern.findall(text)
    unique = int(unique_matches[0]) if unique_matches else None

    return views, unique  # Возвращаем оба числа

# Добавляем данные в список
def data_append(data, name, value):
    data.append(name)
    data.append(value)

# Заменяем значения для столбца "Санузел"
def replace_values(value):
    if isinstance(value, str):  # Проверяем, что значение — строка
        if value == "В\xa0доме":
            return 1
        elif "В\xa0доме" in value:
            return int(value.split()[0])  # Извлекаем число перед "В доме"
    return value  # Оставляем значение без изменений

# Ставим галочку для дополнительных категорий
def check_object(cell, object_name):
    if cell == "-":
        return "-"
    elif object_name in cell.split("\n"):
        return "✓"
    else:
        return "-"

In [5]:
# Классы для значений и наименований
p_class_value = 'a10a3f92e9--color_text-primary-default--vSRPB a10a3f92e9--lineHeight_6u--cedXD a10a3f92e9--fontWeight_normal--JEG_c a10a3f92e9--fontSize_16px--QNYmt a10a3f92e9--display_block--KYb25 a10a3f92e9--text--b2YS3 a10a3f92e9--text_letterSpacing__0--CNLlz'
p_class_text = 'a10a3f92e9--color_gray60_100--r_axa a10a3f92e9--lineHeight_6u--cedXD a10a3f92e9--fontWeight_normal--JEG_c a10a3f92e9--fontSize_16px--QNYmt a10a3f92e9--display_block--KYb25 a10a3f92e9--text--b2YS3 a10a3f92e9--text_letterSpacing__0--CNLlz'

In [6]:
try:
    df_links = pd.read_excel(r'C:\Users\trombee\Desktop\Jupyter Notebooks\Cian\links_3.xlsx', header=None)
except:
    df_links = pd.read_excel(r'C:\Users\trombee\Desktop\Cian\links.xlsx', header=None)

URL_list = df_links[0].to_list()
print(len(URL_list))
URL_list

200


['https://www.cian.ru/sale/suburban/309452671',
 'https://www.cian.ru/sale/suburban/318175909',
 'https://www.cian.ru/sale/suburban/318175919',
 'https://www.cian.ru/sale/suburban/318175908',
 'https://www.cian.ru/sale/suburban/318175924',
 'https://www.cian.ru/sale/suburban/318175923',
 'https://www.cian.ru/sale/suburban/315408264',
 'https://www.cian.ru/sale/suburban/318175913',
 'https://www.cian.ru/sale/suburban/318781295',
 'https://www.cian.ru/sale/suburban/307494361',
 'https://www.cian.ru/sale/suburban/314436651',
 'https://www.cian.ru/sale/suburban/316085731',
 'https://www.cian.ru/sale/suburban/314650461',
 'https://www.cian.ru/sale/suburban/314782124',
 'https://www.cian.ru/sale/suburban/319916570',
 'https://www.cian.ru/sale/suburban/318377630',
 'https://www.cian.ru/sale/suburban/300575480',
 'https://www.cian.ru/sale/suburban/313793998',
 'https://www.cian.ru/sale/suburban/309452623',
 'https://www.cian.ru/sale/suburban/312517175',
 'https://www.cian.ru/sale/suburban/3094

In [7]:
data_soup = []

In [8]:
for i in range(len(URL_list)):
    print(i)
    driver = webdriver.Chrome()
    driver.get(URL_list[i])
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    data_soup.append(soup)
    driver.quit()
    #time.sleep(5)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199


In [9]:
data_dict_template = {
    "Изменение стоимости ИТОГ" : None,
    "% изменения стоимости" : None,
    "Цена без ЗУ ИТОГ" : None,
    "Цена за 1 кв.м. ИТОГ" : None,
    "Цена домовладения актуальная" : None,
    "Ссылка" : None,
    "Цена домовладения СТАРТ" : None,
    "Состояние дома" : None,
    "Цена за 1 кв.м. СТАРТ" : None,
    "Цена за 1 кв.м. без ЗУ СТАРТ" : None,
    "Площадь дома" : None,
    "Площадь" : None,
    "Год постройки" : None,
    "Количество этажей" : None,
    "Количество спален" : None,
    "Санузел" : None,
    "Кол-во дней в экспозиции" : None,
    "Кол-во просмотров всего" : None,
    "Среднее кол-во просмотров в 1 день" : None,
    "Среднее в 1 день уникальных" : None,
    "Кол-во уникальных" : None,
    "Дата выхода в рекламу" : None,
    "Стоимость 1 сотки земли в этом районе" : None,
    "Цена ЗУ" : None,
    "Цена без ЗУ" : None,
    "Газ" : None,
    "Канализация" : None,
    "Водоснабжение" : None,
    "Категория земель" : None,
    "Статус участка" : None,
    "Материал дома" : None,
    "Шоссе" : None,
    "МКАД" : None,
    "КП" : None,
    "Дополнительно" : None,
    "Терраса" : None,
    "Гараж" : None,
    "Баня" : None,
    "Бассейн" : None,
    "Охрана" : None,
    "Сауна" : None,
    "Погреб" : None,
    "Агентство недвижимости" : None
}

In [10]:
column_order = [
    "Изменение стоимости ИТОГ",
    "% изменения стоимости",
    "Цена без ЗУ ИТОГ",
    "Цена за 1 кв.м. ИТОГ",    
    "Ссылка",
    "Цена домовладения актуальная",
    "Цена домовладения СТАРТ",
    "Площадь дома",
    "Площадь",
    "Кол-во дней в экспозиции",
    "Кол-во просмотров всего",
    "Среднее кол-во просмотров в 1 день",
    "Среднее в 1 день уникальных",
    "Кол-во уникальных",
    "Состояние дома",
    "Цена за 1 кв.м. СТАРТ",
    "Цена за 1 кв.м. без ЗУ СТАРТ",
    "Год постройки",
    "Количество этажей",
    "Количество спален",
    "Санузел",
    "Дата выхода в рекламу",
    "Стоимость 1 сотки земли в этом районе",
    "Цена ЗУ",
    "Цена без ЗУ",
    "Газ",
    "Канализация",
    "Водоснабжение",
    "Категория земель",
    "Статус участка",
    "Материал дома",
    "Шоссе",
    "МКАД",
    "КП",
    "Дополнительно",
    "Терраса",
    "Гараж",
    "Баня",
    "Бассейн",
    "Охрана",
    "Сауна",
    "Погреб",
    "Агентство недвижимости"
]

In [11]:
all_data_df = pd.DataFrame()

for i in range(len(data_soup)):
    print(i)
    data_dict = data_dict_template.copy()
    data = []
    try:
    # Находим все элементы с названиями пунктов и значениями
    # Убедитесь, что 'data_soup[0]' указывает на нужный блок
        name_elements = data_soup[i].find_all('p', class_=p_class_text)  # Элементы с названиями пунктов
        value_elements = data_soup[i].find_all('p', class_=p_class_value)  # Элементы со значениями
        
        # Проверяем, что длины списков name_elements и value_elements совпадают
        # Это важно, чтобы избежать ошибок из-за несоответствия количества пунктов и значений
        if len(name_elements) == len(value_elements):
            for name, value in zip(name_elements, value_elements):
                # Извлекаем текст и добавляем в список
                data.append(name.text.strip())   # Название пункта
                data.append(value.text.strip())  # Значение пункта
        else:
            print("Количество названий и значений не совпадает. Проверьте HTML-разметку.")
    
        #Добавление остальных данных
    
        # Добавляем в список data актуальную стоимость домовладения
        price = data_soup[i].find_all('div', class_='a10a3f92e9--amount--ON6i1')[0].text.strip()
        price = int(''.join(filter(str.isdigit, price)))
        data_append(data, 'Цена домовладения актуальная', price)
    
        # Добавляем Цену дома СТАРТ
        data_price_list = []
        len_prices = len(data_soup[i].find_all('tr', class_='a10a3f92e9--history-event--xUQ_P'))
        for prices in range (len_prices):
            data_price_list.append(data_soup[i].find_all('tr', class_='a10a3f92e9--history-event--xUQ_P')[prices].text.strip())
    
        # Добавления начальной стоимости дома, а также дату установки цены
        try:
            if len(data_price_list) != 0:
                prices_list = []
                for max_price in range(len(data_price_list)):
                    parts = data_price_list[max_price].split()
                    part_1 = parts[2][4:]
                    part_2 = parts[3]
                    part_3 = parts[4]
                    price_str = int(part_1 + part_2 + part_3)
                    prices_list.append(price_str)
        
                    adv_date = parts[0] + ' ' + parts[1] + ' ' + parts[2][:4]
                max_price = prices_list[-1]
            else:
                #price = int(''.join(c for c in price if c.isdigit()))
                max_price = int(''.join(c for c in price if c.isdigit()))
                adv_date = '-'
        except:
            max_price = data_soup[i].find_all('div', class_='a10a3f92e9--amount--ON6i1')[0].text.strip()
            max_price = int(''.join(filter(str.isdigit, max_price)))
            adv_date = '-'
    
        data_append(data, 'Цена домовладения СТАРТ', max_price)
        data_append(data, 'Дата выхода в рекламу', adv_date)
    
        # Агентство недвижимости
        try:
            agency = data_soup[i].find_all('span', class_='a10a3f92e9--color_current_color--KRvSV a10a3f92e9--color_current_color--MqB6f a10a3f92e9--lineHeight_6u--cedXD a10a3f92e9--fontWeight_bold--BbhnX a10a3f92e9--fontSize_18px--GiRta a10a3f92e9--display_block--KYb25 a10a3f92e9--text--b2YS3')[0].text.strip()
        except:
            agency = '-'
    
        data_append(data, 'Агентство недвижимости', agency)
    
        # Количество просмотров всего
        views, unique = extract_numbers(data_soup[i].find_all('div', class_='a10a3f92e9--item--qLmCs')[0].text.strip())
        try:
            total_views = int(''.join(views))
        except:
            total_views = views

        try:
            total_unique = int(''.join(unique))
        except:
            total_unique = unique
        mean_views = '-'
        mean_unique = '-'
    
        data_append(data, 'Кол-во просмотров всего', total_views)
        data_append(data, 'Среднее кол-во просмотров в 1 день', mean_views)
        data_append(data, 'Среднее в 1 день уникальных', mean_unique)
        data_append(data, 'Кол-во уникальных', total_unique)
        
    
        # Вывод наименований шоссе
        highways = data_soup[i].find_all('ul', class_='a10a3f92e9--highways--CfZhE')[0].text.strip()
        # result = []
        match = re.findall(r'([А-Яа-я]+ шоссе)', highways)
        highways_name = []   
        result_1 = ''
    
        for cnt in range(len(match)):
            if cnt == 0:
                result_1 += match[cnt]
                result_1 += ', '
            elif cnt == 1:
                result_1 += match[cnt][4:]
            else:
                result_1 += ', '
                result_1 += match[cnt]
        highways_name.append(result_1)
    
        for k in range(len(highways_name)):
            highways_name[k] = highways_name[k].replace(' шоссе', '')
            
        highways_name = str(''.join(highways_name))
    
        data_append(data, 'Шоссе', highways_name)
    
        # Самое низкое число км до МКАД
        numbers = re.findall(r'\d+', highways)
        # Преобразуем найденные числа в целые числа
        numbers = [int(num) for num in numbers]
        # Находим минимальное число в списке
        try:
            min_number_mcad = min(numbers)
        except:
            min_number_mcad = '-'
    
        data_append(data, 'МКАД', min_number_mcad)
        
        # Наименое коттеджного поселка
        village_count = []
        village_count_total = []
        soup = data_soup[i].find_all('a', class_='a10a3f92e9--address--SMU25')
        for j in range(len(soup)):
            village_count.append(soup[j].text.strip())
        village_count_total.append(village_count)
        
        village_name = []
        for item in village_count_total:
            # Проверяем, есть ли в элементе списка "коттеджный поселок", "садовое товарищество" или "СНТ"
            for vil_item in item:
                if 'коттеджный поселок' in vil_item or 'садовое товарищество' in vil_item or 'СНТ' in vil_item:
                    village_name.append(vil_item)
                    break
            else:
                # Если не нашли, добавляем прочерк
                village_name.append('-')
        village_name = str(''.join(village_name))
    
        data_append(data, 'КП', village_name)
        data_append(data, 'Ссылка', URL_list[i])
    
        # Добавление площади дома
        square = data_soup[i].find_all('p', class_=p_class_value)[0].text.strip()
        data_append(data, 'Площадь дома', square)
    
        # Преобразуем список data в словарь
        for fld in range(0, len(data), 2):
            field = data[fld]          # название поля
            value = data[fld + 1]       # значение поля
            
            # Проверяем, если поле есть в словаре, добавляем значение
            if field in data_dict:
                data_dict[field] = value
    
        all_data_df = pd.concat([all_data_df, pd.DataFrame([data_dict])], ignore_index=True)
        all_data_df = pd.DataFrame(all_data_df)[column_order]
    except:
        print('Ошибка')

all_data_df = all_data_df.fillna('-')
all_data_df = all_data_df.replace('\xa0сот.', '', regex=True)
all_data_df = all_data_df.replace('\xa0м²', '', regex=True)
all_data_df['Санузел'] = all_data_df['Санузел'].apply(replace_values)

# Разделяем столбец "Дополнительно" на отдельные категории
objects = ["Терраса", "Гараж", "Баня", "Бассейн", "Охрана", "Сауна", "Погреб"]
for obj in objects:
    all_data_df[obj] = all_data_df["Дополнительно"].apply(lambda x: check_object(x, obj))
all_data_df = all_data_df.drop("Дополнительно", axis=1)

all_data_df['Дата выхода в рекламу'] = all_data_df['Дата выхода в рекламу'].str.replace('мая', 'май')
    
all_data_df

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
Ошибка
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199


,Изменение стоимости ИТОГ,% изменения стоимости,Цена без ЗУ ИТОГ,Цена за 1 кв.м. ИТОГ,Ссылка,Цена домовладения актуальная,Цена домовладения СТАРТ,Площадь дома,Площадь,Кол-во дней в экспозиции,...,МКАД,КП,Терраса,Гараж,Баня,Бассейн,Охрана,Сауна,Погреб,Агентство недвижимости
0,-,-,-,-,https://www.cian.ru/sale/suburban/309452671,38744500,38744500,150,6,-,...,32,Резиденция Булатово коттеджный поселок,-,-,-,-,-,-,-,-
1,-,-,-,-,https://www.cian.ru/sale/suburban/318175909,27500000,27500000,260,8,-,...,30,Монреаль коттеджный поселок,-,-,-,-,-,-,-,-
2,-,-,-,-,https://www.cian.ru/sale/suburban/318175919,37000000,37000000,220,8,-,...,30,Монреаль коттеджный поселок,-,-,-,-,-,-,-,-
3,-,-,-,-,https://www.cian.ru/sale/suburban/318175908,32000000,32000000,260,8,-,...,30,Монреаль коттеджный поселок,-,-,-,-,-,-,-,-
4,-,-,-,-,https://www.cian.ru/sale/suburban/318175924,36500000,36500000,310,10,-,...,30,Монреаль коттеджный поселок,-,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,-,-,-,-,https://www.cian.ru/sale/suburban/319906056,35100000,35100000,480,22,-,...,41,-,-,-,-,-,-,-,-,-
195,-,-,-,-,https://www.cian.ru/sale/suburban/319234258,29800000,29800000,276,12,-,...,35,-,✓,-,-,-,-,-,✓,-
196,-,-,-,-,https://www.cian.ru/sale/suburban/308561923,35000000,35000000,148,10,-,...,41,-,✓,✓,-,-,✓,-,-,-
197,-,-,-,-,https://www.cian.ru/sale/suburban/316543952,35000000,35000000,300,"15,5",-,...,37,Никольские Озера коттеджный поселок,✓,✓,-,-,✓,-,-,-


In [12]:
all_data_df.to_excel(r'C:\Users\trombee\Desktop\Cian\output_data.xlsx', index=False)